LLM Output Cleaning + Dataset Cleaning and Merging

In [1]:
import pandas as pd
import re
import json

In [3]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        df = pd.read_csv(f)
    return df

In [4]:
def extract_json_from_text(text):
    if pd.isna(text):
        return None
    text = str(text)

    fenced_matches = re.findall(r'```(?:json)?\s*({.*?})\s*```', text, flags=re.DOTALL)

    plain_matches = re.findall(r'({\s*"sentiment_score"\s*:\s*.*?})', text, flags=re.DOTALL)

    all_matches = fenced_matches + plain_matches

    if not all_matches:
        return None

    json_text = all_matches[-1].strip()
    json_text = re.sub(r'[\n\r\t]+', ' ', json_text)

    try:
        return json.loads(json_text)
    except json.JSONDecodeError:
        try:
            json_text = re.sub(r',\s*}', '}', json_text)
            json_text = re.sub(r',\s*]', ']', json_text)
            return json.loads(json_text)
        except Exception:
            return None
    except Exception:
        return None


In [5]:
def clean_llm_output(file_path, save_path):
    df = load_data(file_path)
    df["parsed_json"] = df["reason"].apply(extract_json_from_text)
    df["llm_score"] = df["parsed_json"].apply(lambda x: x.get("sentiment_score") if isinstance(x, dict) else None)
    df["llm_predicted_direction"] = df["parsed_json"].apply(lambda x: x.get("predicted_direction") if isinstance(x, dict) else None)
    df["llm_reason"] = df["parsed_json"].apply(lambda x: x.get("reason") if isinstance(x, dict) else None)

    df = df.drop(columns=["parsed_json", "reason", "sentiment_score", "predicted_direction","sentiment_label"])
    if save_path:
        with open(save_path, "w", encoding="utf-8") as f:
            df.to_csv(f)
        print(f"Saved cleaned file at : {save_path}")

    total_rows = len(df)
    valid_rows = df["llm_score"].notna().sum()
    invalid_rows = total_rows - valid_rows

    print("✅ Cleaning complete!")
    print(f"📊 Total rows: {total_rows}")
    print(f"✅ Successfully parsed: {valid_rows}")
    print(f"⚠️ Skipped / hallucinated: {invalid_rows}")

    print("Missing data samples:")
    missing = df[df["llm_score"].isna()]
    print(missing[["llm_score"]])
    return df

In [7]:
df_rel = clean_llm_output("D:/5TH SEM/stock_sentiment_project/data/clean_reliance_with_sentiments_with_llm_sentiments.csv","D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/rel2.csv")


Saved cleaned file at : D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/rel2.csv
✅ Cleaning complete!
📊 Total rows: 203
✅ Successfully parsed: 193
⚠️ Skipped / hallucinated: 10
Missing data samples:
     llm_score
35         NaN
37         NaN
66         NaN
102        NaN
128        NaN
149        NaN
162        NaN
182        NaN
187        NaN
200        NaN


In [10]:
df_rel.head()

,author,title,description,source,category,language,country,published_at,combined_text,company,textblob_score,vader_score,finbert_score,llm_score,llm_predicted_direction,llm_reason
0,Aishwarya Patil,"Jio Delivers Speed, Airtel Wins On Voice—TRAI ...","Jio Delivers Speed, Airtel Wins On Voice—TRAI ...",Bloomberg | Latest And Live Business,business,en,us,2025-07-01,"Jio Delivers Speed, Airtel Wins On Voice—TRAI ...",Reliance,0.300000,0.8126,0.0,0.85,Up,TRAI's drive test reveals significant gaps acr...
1,Prajwal Jayaraj,'Not A Business Where Deep Pockets Mean A Larg...,'Not A Business Where Deep Pockets Mean A Larg...,Bloomberg | Latest And Live Business,business,en,us,2025-07-02,'Not A Business Where Deep Pockets Mean A Larg...,Reliance,-0.032738,0.0000,0.0,0.50,Down,The article suggests that Jio-Blackrock's acqu...
2,Vivek Singh,Jio-Krafton Partnership: A “Milestone” for Ind...,Recently Jio-Krafton Partnership introduces ne...,Insidesport,sports,en,us,2025-07-03,Jio-Krafton Partnership: A “Milestone” for Ind...,Reliance,0.128788,0.0000,0.0,0.80,Up,The partnership between Jio and Krafton is see...
3,NaN,"Reliance Retail invests in UK’s FaceGym, to br...","Reliance Retail invests in UK’s FaceGym, to br...",Business Line,business,en,us,2025-07-03,"Reliance Retail invests in UK’s FaceGym, to br...",Reliance,0.000000,0.8225,0.0,0.50,Up,Reliance Retail's investment in FaceGym in the...
4,Bloomberg News,Reliance To Spin Off Consumer Goods Brands Ahe...,Reliance To Spin Off Consumer Goods Brands Ahe...,Bloomberg | Latest And Live Business,business,en,us,2025-07-03,Reliance To Spin Off Consumer Goods Brands Ahe...,Reliance,0.000000,0.0000,0.0,0.60,Up,"The news mentions Reliance Jio, a subsidiary o..."


In [8]:
df_sbi = clean_llm_output("D:/5TH SEM/stock_sentiment_project/data/clean_sbi_with_sentiments_with_llm_sentiments.csv","D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/sbi2.csv")

Saved cleaned file at : D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/sbi2.csv
✅ Cleaning complete!
📊 Total rows: 526
✅ Successfully parsed: 517
⚠️ Skipped / hallucinated: 9
Missing data samples:
     llm_score
49         NaN
84         NaN
94         NaN
114        NaN
203        NaN
232        NaN
296        NaN
316        NaN
391        NaN


In [17]:
df_sbi.head()

,author,title,description,source,category,language,country,published_at,combined_text,company,textblob_score,vader_score,finbert_score,llm_score,llm_predicted_direction,llm_reason
0,Subhana Shaikh,RBI Monetary Policy: SBI Research Bets On Shar...,RBI Monetary Policy: SBI Research Bets On Shar...,Bloomberg | Latest And Live Business,business,en,us,2025-06-03,RBI Monetary Policy: SBI Research Bets On Shar...,SBI,-0.12500,-0.4939,0.0,0.5,Down,RBI Monetary Policy: SBI Research Bets On Shar...
1,NaN,"Day Trading Guide for June 4, 2025: Intraday s...",Day Trading Guide gives you the key intraday s...,Business Line,business,en,us,2025-06-04,"Day Trading Guide for June 4, 2025: Intraday s...",SBI,-0.05625,0.7717,0.0,0.5,Neutral,The article provides general intraday supports...
2,Personal Finance Desk,Latest PSU Banks FD Rates: Check SBI vs PNB vs...,Latest PSU Banks FD Rates: Check SBI vs PNB vs...,Bloomberg | Latest And Live Business,business,en,us,2025-06-04,Latest PSU Banks FD Rates: Check SBI vs PNB vs...,SBI,0.50000,0.7184,0.0,0.5,Neutral,The article provides general information about...
3,NaN,"Day Trading Guide for June 5, 2025: Intraday s...",Day Trading Guide gives you the key intraday s...,Business Line,business,en,us,2025-06-05,"Day Trading Guide for June 5, 2025: Intraday s...",SBI,-0.05625,0.7717,0.0,0.6,Up,"Intraday supports, resistances for Nifty50 sto..."
4,NaN,"Bajaj Finserv block deal: SBI MF, Goldman Sach...",Entities buy 2.86 crore shares in Bajaj Finser...,Business Line,business,en,us,2025-06-06,"Bajaj Finserv block deal: SBI MF, Goldman Sach...",SBI,-0.15000,-0.1779,0.0,0.6,Up,"Bajaj Finserv block deal: SBI MF, Goldman Sach..."


In [9]:
df_inf = clean_llm_output("D:/5TH SEM/stock_sentiment_project/data/clean_infosys_with_sentiments_with_llm_sentiments.csv","D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/inf2.csv")

Saved cleaned file at : D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/inf2.csv
✅ Cleaning complete!
📊 Total rows: 303
✅ Successfully parsed: 298
⚠️ Skipped / hallucinated: 5
Missing data samples:
     llm_score
14         NaN
66         NaN
128        NaN
183        NaN
281        NaN


In [18]:
df_inf.head()

,author,title,description,source,category,language,country,published_at,combined_text,company,textblob_score,vader_score,finbert_score,llm_score,llm_predicted_direction,llm_reason
0,Investing.com,infosys announces 44th annual general meeting ...,infosys announces 44th annual general meeting ...,Investing.com | Stock Market Quotes &amp; Fina...,business,en,us,2025-06-02,infosys announces 44th annual general meeting ...,Infosys,0.050000,0.0000,0.000000,0.00,Down,Infosys announces 44th annual general meeting...
1,Haripriya Sureban,Infosys CEO Salil Parekh Gets 22% Pay Hike In ...,Infosys CEO Salil Parekh Gets 22% Pay Hike In ...,Bloomberg | Latest And Live Business,business,en,us,2025-06-02,Infosys CEO Salil Parekh Gets 22% Pay Hike In ...,Infosys,0.000000,-0.2023,0.812474,0.50,Neutral,The 22% pay hike in FY25 is a positive develop...
2,Roger Kehrt,Infosys BPM Unveils AI Agents for Accounts Pay...,"Infosys BPM unveiled AI agents on May 30, 2025...",webpronews,general,en,us,2025-06-02,Infosys BPM Unveils AI Agents for Accounts Pay...,Infosys,0.250000,0.7845,0.815225,0.60,Up,"Infosys BPM unveiled AI agents on May 30, 2025..."
3,ABMN Staff,UBS AM a distinct business unit of UBS ASSET M...,UBS AM a distinct business unit of UBS ASSET M...,americanbankingnews,general,en,us,2025-06-02,UBS AM a distinct business unit of UBS ASSET M...,Infosys,0.114286,0.5670,-0.720634,-0.75,Down,UBS AM a distinct business unit of UBS ASSET M...
4,Ananya Chaudhuri,Stock Market Live: GIFT Nifty Implies Lower Op...,Stock Market Live: GIFT Nifty Implies Lower Op...,Bloomberg | Latest And Live Business,business,en,us,2025-06-02,Stock Market Live: GIFT Nifty Implies Lower Op...,Infosys,0.068182,0.9130,-0.869344,0.50,Neutral,The article discusses the lower open of the GI...


merging stock and news data

In [28]:
import pandas as pd
import os

def merge_datasets(news_path, stock_path, save_path=None):
    # === 1️⃣ Load data ===
    news_df = pd.read_csv(news_path)
    stock_df = pd.read_csv(stock_path)

    # === 2️⃣ Rename + convert dates ===
    stock_df.rename(columns={'Date': 'Trading_Date'}, inplace=True)
    news_df.rename(columns={'published_at': 'News_Date'}, inplace=True)

    # Convert both to datetime, strip timezone info
    stock_df['Trading_Date'] = pd.to_datetime(stock_df['Trading_Date'], errors='coerce').dt.tz_localize(None)
    news_df['News_Date'] = pd.to_datetime(news_df['News_Date'], errors='coerce').dt.tz_localize(None)

    # Drop rows with missing dates
    news_df.dropna(subset=['News_Date'], inplace=True)
    stock_df.dropna(subset=['Trading_Date'], inplace=True)

    # === 3️⃣ Sort by date ===
    stock_df = stock_df.sort_values('Trading_Date').reset_index(drop=True)
    news_df = news_df.sort_values('News_Date').reset_index(drop=True)

    # === 4️⃣ Map each news → next available trading day ===
    trading_dates = stock_df['Trading_Date'].tolist()
    mapped_dates = []
    for date in news_df['News_Date']:
        next_dates = [d for d in trading_dates if d >= date + pd.Timedelta(days=1)]
        mapped_dates.append(next_dates[0] if next_dates else pd.NaT)

    news_df['Matched_Trading_Date'] = mapped_dates

    # 🔧 Drop rows with no mapped trading date (no stock day after news)
    news_df.dropna(subset=['Matched_Trading_Date'], inplace=True)

    # === 5️⃣ Merge (left=news, right=stock) ===
    merged_df = pd.merge_asof(
        news_df.sort_values('Matched_Trading_Date'),
        stock_df.sort_values('Trading_Date'),
        left_on='Matched_Trading_Date',
        right_on='Trading_Date',
        direction='nearest',  # match to the closest next trading day
        tolerance=pd.Timedelta(days=5)
    )

    # === 6️⃣ Clean up ===
    merged_df.rename(columns={'company': 'Company_Name'}, inplace=True)
    merged_df.drop(columns=['Matched_Trading_Date'], inplace=True, errors='ignore')

    # === 7️⃣ Handle missing numeric fields ===
    merged_df.fillna(method='ffill', inplace=True)
    merged_df['Target'] = merged_df['Target'].replace(-1, 0)

    # === 8️⃣ Save merged ===
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        merged_df.to_csv(save_path, index=False)
        print(f"✅ Merged dataset saved to: {save_path}")

    print(f"✅ Final merged shape: {merged_df.shape}")
    print(merged_df[['News_Date', 'Trading_Date']].head(10))
    return merged_df


In [29]:
df_sbi_merged = merge_datasets("D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/news/sbi2.csv",
                               "D:/5TH SEM/stock_sentiment_project/data/preprocessed/stock/clean_sbi_stock.csv",
                               "D:/5TH SEM/stock_sentiment_project/data/merged/sbi_data.csv")

✅ Merged dataset saved to: D:/5TH SEM/stock_sentiment_project/data/merged/sbi_data.csv
✅ Final merged shape: (524, 37)
   News_Date        Trading_Date
0 2025-06-03 2025-06-04 18:30:00
1 2025-06-04 2025-06-05 18:30:00
2 2025-06-04 2025-06-05 18:30:00
3 2025-06-05 2025-06-08 18:30:00
4 2025-06-06 2025-06-08 18:30:00
5 2025-06-06 2025-06-08 18:30:00
6 2025-06-06 2025-06-08 18:30:00
7 2025-06-06 2025-06-08 18:30:00
8 2025-06-06 2025-06-08 18:30:00
9 2025-06-07 2025-06-08 18:30:00


C:\Users\Hemant\AppData\Local\Temp\ipykernel_2704\3486068710.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)


In [30]:
df_rel_merged = merge_datasets("D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/news/rel2.csv",
                               "D:/5TH SEM/stock_sentiment_project/data/preprocessed/stock/clean_reliance_stock.csv",
                               "D:/5TH SEM/stock_sentiment_project/data/merged/reliance_data.csv")

✅ Merged dataset saved to: D:/5TH SEM/stock_sentiment_project/data/merged/reliance_data.csv
✅ Final merged shape: (195, 37)
   News_Date        Trading_Date
0 2025-07-01 2025-07-02 18:30:00
1 2025-07-02 2025-07-03 18:30:00
2 2025-07-03 2025-07-06 18:30:00
3 2025-07-03 2025-07-06 18:30:00
4 2025-07-03 2025-07-06 18:30:00
5 2025-07-04 2025-07-06 18:30:00
6 2025-07-06 2025-07-07 18:30:00
7 2025-07-07 2025-07-08 18:30:00
8 2025-07-07 2025-07-08 18:30:00
9 2025-07-07 2025-07-08 18:30:00


C:\Users\Hemant\AppData\Local\Temp\ipykernel_2704\3486068710.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)


In [31]:
df_inf_merged = merge_datasets("D:/5TH SEM/stock_sentiment_project/data/with_sentiment_score/news/inf2.csv",
                               "D:/5TH SEM/stock_sentiment_project/data/preprocessed/stock/clean_infosys_stock.csv",
                               "D:/5TH SEM/stock_sentiment_project/data/merged/infosys_data.csv")

✅ Merged dataset saved to: D:/5TH SEM/stock_sentiment_project/data/merged/infosys_data.csv
✅ Final merged shape: (303, 37)
   News_Date        Trading_Date
0 2025-06-02 2025-06-03 18:30:00
1 2025-06-02 2025-06-03 18:30:00
2 2025-06-02 2025-06-03 18:30:00
3 2025-06-02 2025-06-03 18:30:00
4 2025-06-02 2025-06-03 18:30:00
5 2025-06-02 2025-06-03 18:30:00
6 2025-06-02 2025-06-03 18:30:00
7 2025-06-03 2025-06-04 18:30:00
8 2025-06-03 2025-06-04 18:30:00
9 2025-06-04 2025-06-05 18:30:00


C:\Users\Hemant\AppData\Local\Temp\ipykernel_2704\3486068710.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)


In [1]:
import pandas as pd
import glob

# Example: merge all CSVs in a folder
csv_files = glob.glob("D:/5TH SEM/stock_sentiment_project/data/merged/*.csv")  # e.g., "data/*.csv"
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    # Drop any column that starts with 'Unnamed' (usually old indices)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    dfs.append(df)

# Merge all into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)
print("Merged shape:", merged_df.shape)


Merged shape: (1022, 35)


In [3]:
merged_df.to_csv("D:/5TH SEM/stock_sentiment_project/data/merged/merged_data.csv", index=False)
print("Saved merged CSV as merged_data.csv")

Saved merged CSV as merged_data.csv
